<a href="https://colab.research.google.com/github/BriouaNorElHouda/py/blob/main/lanMS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import zipfile
import requests
import os
import io
import cv2
import shutil
from PIL import Image
import tensorflow as tf
from skimage.transform import resize
from tensorflow import keras
from keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from keras.models import Sequential
import keras
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten,Conv2D , MaxPooling2D
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.losses import CategoricalCrossentropy
from tqdm import tqdm
from keras.applications import vgg16
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model

from tensorflow.keras.applications import VGG16
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.utils import plot_model
import time
import rasterio
from rasterio.plot import show, show_hist,  reshape_as_image


In [ ]:
!pip install rasterio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.5/21.5 MB 45.4 MB/s eta 0:00:00


In [ ]:
labels = {
    'AnnualCrop': 0,
    'Forest': 1,
    'HerbaceousVegetation': 2,
    'Highway': 3,
    'Industrial': 4,
    'Pasture': 5,
    'PermanentCrop': 6,
    'Residential': 7,
    'River': 8,
    'SeaLake': 9,
}

In [ ]:
zip_path = '/content/drive/MyDrive/Colab Notebooks/EuroSatMS.zip'

extracted_path = "/content/EuroSatDS"
with zipfile.ZipFile(zip_path, 'r') as z:
    z.extractall(extracted_path)

data = []

subclass = sorted(os.listdir(src_path))

def create_data():
    for sclass in subclass:
        path = os.path.join(src_path, sclass)
        class_num = subclass.index(sclass)
        for img in tqdm(os.listdir(path)):
            img_array = cv2.imread(os.path.join(path, img), cv2.IMREAD_COLOR)
            new_array = cv2.resize(img_array, (64, 64))
            data.append([new_array, class_num])

create_data()

print("Total samples in the dataset:", len(data))

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/Colab Notebooks/EuroSatMS.zip'

In [ ]:
X = []
y = []

for features, label in data:
    X.append(features)
    y.append(label)


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

class MyDataGenerator(keras.utils.Sequence):
    def __init__(self, X, y, batch_size):
        self.X = X
        self.y = y
        self.batch_size = batch_size
        self.num_samples = len(X)

    def __len__(self):
        return int(np.ceil(self.num_samples / self.batch_size))

    def __getitem__(self, index):
        start = index * self.batch_size
        end = min((index + 1) * self.batch_size, self.num_samples)

        batch_X = self.X[start:end]
        batch_y = self.y[start:end]

        batch_X = np.array(batch_X).reshape(-1,64, 64, 13) / 255.0
        batch_y = np.array(batch_y)

        return batch_X, batch_y

batch_size = 64
train_generator = MyDataGenerator(X_train, y_train, batch_size)
test_generator = MyDataGenerator(X_test, y_test, batch_size)

In [ ]:
NAME="MS115"

model=Sequential(Input(shape=(100,)))

model.add(Conv2D(32, kernel_size=(3, 3),  activation='relu',  input_shape=(64, 64, 13)))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(256, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))


model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(512, activation='relu'))
model.add(Dense(10, activation='softmax'))

tensorboard= TensorBoard(log_dir="logs\{}".format(NAME))

model.compile(loss='sparse_categorical_crossentropy',
              optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
              metrics=['accuracy'])


history = model.fit(train_generator, epochs=100,batch_size=64, validation_data=test_generator,callbacks=[tensorboard])

In [ ]:
NAME="MS295"
​
model=Sequential()
​
​
# Initial Convolutional layer
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(64, 64, 13), padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))
​
# Convolutional layers
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))
​
model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))
​
model.add(Flatten())
model.add(Dropout(0.3))
model.add(Dense(512, activation='relu'))
model.add(Dense(10, activation='softmax'))
​
tensorboard= TensorBoard(log_dir="logs\{}".format(NAME))
​
model.compile(loss='sparse_categorical_crossentropy',
              optimizer=tf.keras.optimizers.Adam(learning_rate=0.00001),
              metrics=['accuracy'])
​
​
history = model.fit(train_generator, epochs=120,batch_size=32, validation_data=test_generator,callbacks=[tensorboard])

In [ ]:
import tensorflow.keras
import numpy as np
import tensorflow as tf

def modify_model(model, new_input_shape=(None, 64, 64, 13)):
    model.layers[0].batch_input_shape = new_input_shape
    new_input = Input(shape=new_input_shape)
    new_output = model.layers[-2](new_input)
    new_model = Model(inputs=new_input, outputs=new_output)

    return new_model

In [ ]:
#VGG16

conv_base=vgg16.VGG16(weights='imagenet',include_top=False,input_shape=(64, 64,3))

conv_base_MS =  modify_model(conv_base, new_input_shape = (64, 64, 13))
conv_base_MS.summary()

ValueError: Exception encountered when calling layer "block5_conv3" (type Conv2D).

Depth of input (13) is not a multiple of input depth of filter (512) for '{{node block5_conv3/Conv2D}} = Conv2D[T=DT_FLOAT, data_format="NHWC", dilations=[1, 1, 1, 1], explicit_paddings=[], padding="SAME", strides=[1, 1, 1, 1], use_cudnn_on_gpu=true](Placeholder, block5_conv3/Conv2D/ReadVariableOp)' with input shapes: [?,64,64,13], [3,3,512,512].

Call arguments received by layer "block5_conv3" (type Conv2D):
  • inputs=tf.Tensor(shape=(None, 64, 64, 13), dtype=float32)

In [ ]:
NAME="MSVGG"

model=Sequential()
model.add(convggMS)
model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(512, activation='relu'))
model.add(Dense(10, activation='softmax'))

tensorboard= TensorBoard(log_dir="logs\{}".format(NAME))

model.compile(loss='sparse_categorical_crossentropy',
              optimizer=tf.keras.optimizers.Adam(learning_rate=0.00001),
              metrics=['accuracy'])


history = model.fit(train_generator, epochs=100,batch_size=64, validation_data=test_generator,callbacks=[tensorboard])

In [ ]:
#ResNet50
base_model = keras.applications.ResNet50(weights='imagenet', include_top=False, input_shape=(64, 64, 3))
ResNetMS = modify_model(base_model, new_input_shape=(64, 64, 13))

94765736/94765736 [==============================] - 1s 0us/step


ValueError: Exception encountered when calling layer "conv5_block3_add" (type Add).

A merge layer should be called on a list of inputs. Received: inputs=Tensor("Placeholder:0", shape=(None, 64, 64, 13), dtype=float32) (not a list of tensors)

Call arguments received by layer "conv5_block3_add" (type Add):
  • inputs=tf.Tensor(shape=(None, 64, 64, 13), dtype=float32)

In [ ]:
NAME="MSRN"

model=Sequential()
model.add(ResNetMS)
model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(512, activation='relu'))
model.add(Dense(10, activation='softmax'))

tensorboard= TensorBoard(log_dir="logs\{}".format(NAME))

model.compile(loss='sparse_categorical_crossentropy',
              optimizer=tf.keras.optimizers.Adam(learning_rate=0.00001),
              metrics=['accuracy'])


history = model.fit(train_generator, epochs=100,batch_size=64, validation_data=test_generator,callbacks=[tensorboard])

In [ ]:
#Plot Model Loss
plt.subplot(211)
plt.title('Model Loss')
plt.plot(history.history['loss'], color='blue', label='train')
plt.plot(history.history['val_loss'], color='green', label='test')
plt.ylabel('Loss')
plt.xlabel('Epochs')
plt.legend(['Training','Validation'],loc='upper right')


In [ ]:

# Plot Accuracy
plt.subplot(211)
plt.title('Model Accuracy')
plt.plot(history.history['accuracy'], color='blue', label='train')
plt.plot(history.history['val_accuracy'], color='green', label='test')
plt.ylabel('Accuracy')
plt.xlabel('Epochs')
plt.legend(['Training','Validation'],loc='lower right')

In [ ]:
save_path = '/content/drive/MyDrive/Colab Notebooks/MyModels/vggms115.h5'
model.save(save_path)

In [ ]:
model.summary()

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

# Predict the test set
y_pred = model.predict(test_generator)
y_pred_classes = np.argmax(y_pred, axis=1)

# Create a confusion matrix
cm = confusion_matrix(y_test, y_pred_classes)
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=labels, yticklabels=labels)
plt.xlabel("Predicted Class")
plt.ylabel("True Class")
plt.title("Confusion Matrix")
plt.show()